# UPS MV Framework

### Import Libraries

In [ ]:
from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics import summary
from AssetAllocation.reporting import reports as rp, plots, formatter as fmt
from ipywidgets import interact, interact_manual
import numpy as np

PLAN = 'IBT'

### Compute inputs (return, volatility and correlation) for plan

In [ ]:
#get return
mv_inputs_dict = dm.get_mv_inputs_data('inputs_test.xlsx', plan=PLAN)
mv_inputs = summary.get_mv_inputs(mv_inputs_dict)
#get historical vol and correlation
ts_dict = dm.get_ts_data(plan=PLAN)
ts_outputs = summary.get_ts_output(ts_dict)
pp_inputs = summary.get_pp_inputs(mv_inputs, ts_dict)

### Display plan inputs

In [ ]:
@interact
def display_pp_inputs(inputs=pp_inputs.keys()):
    if inputs == 'corr':
        return plots.draw_heatmap(pp_inputs[inputs], half=False)
    else:
        return fmt.get_ret_vol_styler(pp_inputs[inputs])

### Create plan object that has the relevant variables

In [ ]:
plan = summary.get_plan_params(pp_inputs)

pp_dict = summary.get_pp_dict(plan)
@interact
def display_pp_inputs(variable=pp_dict.keys()):
    if variable == 'Corr' or variable == 'Cov':
        return plots.draw_heatmap(pp_dict[variable], half=False)
    else:
        return fmt.get_ret_vol_styler(pp_dict[variable])

### Define bounds
* Bounds data is stored in **'data/plan_inputs/bounds.xlsx'** by plan

In [ ]:
bnds = dm.get_bounds(plan=PLAN)

#View bounds
df_bnds = dm.pd.DataFrame(bnds, index=plan.symbols, columns=['Lower', 'Upper'])
fmt.get_ret_vol_styler(df_bnds)

### Define contraints to optimize for min and max return

In [ ]:
cons = (
    # sum of Fixed Income Assets >= 50%
    {'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
    #sum of all plan assets (excluding Futures and Hedges) = 2%    
    {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-1]) - x[3] - .02}
)

### Compute and Display MV Efficient Frontier portfolios

In [ ]:
#Get data for MV efficient frontier portfolios
plan.compute_eff_frontier(bnds,cons,num_ports=100)

#Dataframe containing efficient frontier portfolios (return, vol, sharpe and weights)
ports_df = dm.get_ports_df(plan.eff_frontier_trets, plan.eff_frontier_tvols, plan.eff_frontier_tweights, plan.symbols)
fmt.get_port_styler(ports_df)

### Display MV Asset Allocation

In [ ]:
#Asset Allocation Plot
aa_fig = plots.get_aa_fig(ports_df)
aa_fig.show()

### Display MV Efficient Frontier

In [ ]:
#Plotly version of the Efficient Frontier plot
ef_fig = plots.get_ef_fig(ports_df)
ef_fig.show()

### Export data to excel

In [ ]:
#Export Efficient Frontier portfoio data to excel
rp.get_ef_portfolios_report(PLAN+'_ef_portfolios', plan)

In [ ]:
plan.symbols